In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

In [2]:
df = pd.read_csv('../Data/galway_rentals.txt', delimiter='\t')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   price_per_month       395 non-null    int64  
 1   distance_eyre_square  395 non-null    float64
 2   distance_salthill     395 non-null    float64
 3   distance_nuig         395 non-null    float64
 4   distance_gmit         395 non-null    float64
 5   num_bedrooms          395 non-null    int64  
 6   num_bathrooms         395 non-null    int64  
 7   type                  395 non-null    object 
 8   ber                   395 non-null    object 
 9   balcony               395 non-null    object 
 10  floor                 395 non-null    object 
 11  heating               395 non-null    object 
dtypes: float64(4), int64(3), object(5)
memory usage: 37.2+ KB


In [4]:
df.head()

,price_per_month,distance_eyre_square,distance_salthill,distance_nuig,distance_gmit,num_bedrooms,num_bathrooms,type,ber,balcony,floor,heating
0,3700,1.112343,2.458528,0.663391,3.046377,5,3,house,exempt,no,ground,central
1,3640,1.045924,2.687242,1.013547,2.627561,6,3,house,b3,no,ground,central
2,2950,1.876834,2.365351,0.889952,4.098964,4,2,house,c1,no,ground,central
3,2825,1.739580,1.340990,0.973981,4.275886,3,2,apartment,d1,no,second,electric
4,2800,2.446769,1.037478,1.917976,4.984487,4,3,house,c1,no,ground,central


In [5]:
""" Encoding categorical features
Ordinal features:
    ber: a->exempt
    floor: third, second, first, ground

Nominal features: - no order
    type: house, studio, apartment, townhouse
    heating: central, electric, NA
    Balcony: yes, no
"""
ber = ['a', 'b1', 'b2', 'b3', 'c1', 'c2', 'c3', 'd1', 'd2', 'e1', 'e2', 'f', 'g', 'exempt']
floor = ['third', 'second', 'first', 'ground']
ord_encoder = OrdinalEncoder(categories = [ber, floor])
df[['ber', 'floor']] = ord_encoder.fit_transform(df[["ber", 'floor']])

lab_encoder = 

In [6]:
df.head()


,price_per_month,distance_eyre_square,distance_salthill,distance_nuig,distance_gmit,num_bedrooms,num_bathrooms,type,ber,balcony,floor,heating
0,3700,1.112343,2.458528,0.663391,3.046377,5,3,house,13.0,no,3.0,central
1,3640,1.045924,2.687242,1.013547,2.627561,6,3,house,3.0,no,3.0,central
2,2950,1.876834,2.365351,0.889952,4.098964,4,2,house,4.0,no,3.0,central
3,2825,1.739580,1.340990,0.973981,4.275886,3,2,apartment,7.0,no,1.0,electric
4,2800,2.446769,1.037478,1.917976,4.984487,4,3,house,4.0,no,3.0,central
